In [91]:
import logging
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import wandb
from model import UNet, UNetWithGradCAM
from utils.data_loading import BuildingsDataset

In [92]:
model = UNetWithGradCAM(UNet(5, 3))
model.load_state_dict(torch.load("model.pth"))

# Load and preprocess the test dataset
DATA_DIR = "../data/"
x_test_dir = os.path.join(DATA_DIR, "test/")

test_dataset = BuildingsDataset(x_test_dir)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

In [93]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Move model to device
model = model.to(device)

In [106]:
def precision(y_true, y_pred, class_label):
    true_positive = np.sum((y_true == class_label) & (y_pred == class_label))
    false_positive = np.sum((y_true != class_label) & (y_pred == class_label))
    
    if true_positive + false_positive == 0:
        return 0.0
    return true_positive / (true_positive + false_positive)

def recall(y_true, y_pred, class_label):
    true_positive = np.sum((y_true == class_label) & (y_pred == class_label))
    false_negative = np.sum((y_true == class_label) & (y_pred != class_label))
    
    if true_positive + false_negative == 0:
        return 0.0
    return true_positive / (true_positive + false_negative)

def f1_score(y_true, y_pred, class_label):
    p = precision(y_true, y_pred, class_label)
    r = recall(y_true, y_pred, class_label)
    
    if p + r == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

In [122]:
f1_scores = [0.0, 0.0, 0.0]
recall_scores = [0.0, 0.0, 0.0]
precision_scores = [0.0, 0.0, 0.0]

nums = [0, 0, 0]

for image, gt_mask in test_dataset:
    x_tensor = torch.from_numpy(image).to(device).unsqueeze(0)
    x_tensor = x_tensor.permute(0, 3, 1, 2).float().to(device)
    # Predict test image
    pred_mask = model(x_tensor)
    tensor_to_detach = pred_mask[
        0
    ]  # Assuming the tensor you want to detach is the first element of the tuple
    pred_mask_array = tensor_to_detach.detach().squeeze().cpu().numpy()
    pred_mask = np.transpose(pred_mask_array, (1, 2, 0))
    gt_mask = np.argmax(gt_mask, axis=2)
    pred_mask = np.argmax(pred_mask, axis=2)
    classes = np.unique(gt_mask)
    for cl in classes:
        recall_scores[cl] += recall(gt_mask, pred_mask, cl)
        precision_scores[cl] += precision(gt_mask, pred_mask, cl)
        f1_scores[cl] += f1_score(gt_mask, pred_mask, cl)
        nums[cl] += 1

f1_scores = np.array(f1_scores)
recall_scores = np.array(recall_scores)
precision_scores = np.array(precision_scores)
nums = np.array(nums)

In [123]:
print(f"""
    Recall Scores per class:
        {recall_scores / nums}
    Precision Scores per class:
        {precision_scores / nums}
    F1 Scores per class:
        {f1_scores / nums}
""")


    Recall Scores per class:
        [0.99994141 0.18139851 0.16286149]
    Precision Scores per class:
        [0.999675   0.25067889 0.16590563]
    F1 Scores per class:
        [0.99980816 0.19794879 0.15692542]

